## LIMPIEZA Y ORGANIZACIÓN DEL DATA SET

#### 1. Importamos las librerias que nos seran utiles

In [218]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import src.MetodosCleaningAnalysis as cf

#### 2. Importamos los datos

In [138]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [139]:
sdf = pd.DataFrame(sharks)

#### -Como podemos observar, la mayoria de los datos son no numericos...

In [140]:
sdf.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

## HIPOTESIS I

#### - Vamos a quedarnos con las columnas que serán relevantes para el analisis de nuestra primer hipotesis. Para ello, utilizaremos la función "keep_columns".

In [141]:
cf.keep_columns(['Case Number', 'Type', 'Case Number.1', 'Case Number.2'], sdf)

#### -Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres. Para eso, aplicamos un método de numpy que nos devuelve un resultado booleano luego de comparar fila por fila si los valores de dos columnas coinciden o no.

In [142]:
compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)

In [143]:
sdf['equal1'] = compare1

In [144]:
sdf['equal1'].value_counts() 

False    19445
True      6278
Name: equal1, dtype: int64

In [145]:
compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)

In [146]:
sdf['equal2'] = compare2

In [147]:
sdf['equal2'].value_counts() 

False    19425
True      6298
Name: equal2, dtype: int64

In [148]:
compare3 = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [149]:
sdf['equal3'] = compare3

In [150]:
sdf['equal3'].value_counts() 

False    19441
True      6282
Name: equal3, dtype: int64

In [151]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal1               0
equal2               0
equal3               0
dtype: int64

#### - Al comparar las tres columnas entre sí, en donde menos diferencia de valores hay es entre 'Case Number' y 'Case Number .2'. De todas formas, para la cantidad de datos que tenemos la diferencia es mínima, y de las tres, la que menos cantidad de valores null tiene es 'Case Number', por lo tanto para obtener valores sobre la fecha de los ataques nos quedamos con ella, y con 'Type'.

In [152]:
cf.keep_columns(['Case Number', 'Type'], sdf)

#### -Vamos a eliminar los valores que no tengan formato de fecha, ya que nos interesa ver la estacionalidad mensual. Empezamos por los valores "NaN", y luego añadiremos una columna con los valores que solo. Para ello utilizaremos la funcion "filtración".

In [153]:
sdf.dropna(inplace=True)

In [154]:
sdf['Months'] = sdf['Case Number'].apply(cf.filtracion)

#### - Borramos los valores nulos que corresponden a los datos que no tenian registro mensual.

In [155]:
sdf.dropna(inplace=True)

#### - Exportamos el data frame a un archivo csv para luego realizar las visualizaciones.

In [156]:
sdf.to_csv('sdf',index=False)

#### - Ahora construimos un dataframe cuyas columnas contendrán las frecuencias mensuales de los ataques y los meses correspndientes, consideramos todos los tipos de ataque, con decenlace letal o no y renombramos las columnas para emprolijar.

In [157]:
ataques_mensuales = pd.DataFrame(sdf['Months'].value_counts()).reset_index()

In [158]:
c = {'Months': 'Monthly Attacks', 'index':'Month'}
ataques_mensuales = ataques_mensuales.rename(columns=c)

In [159]:
ataques_mensuales = ataques_mensuales.reindex([3, 11, 8, 5, 10, 4, 0, 1, 2, 7, 9, 6])

#### - Exportamos el data frame de ataques mensuales para utilizarlo en las visualizaciones

In [160]:
ataques_mensuales.to_csv('ataques_mensuales',index=False)

## HIPOTESIS II

In [211]:
sdf2= pd.DataFrame(sharks)

#### Vamos a quedarnos solo con las columnas que nos interesan para analizar la H2, 'Species ' , 'Injury' y 'Fatal'

In [212]:
cf.keep_columns(['Injury', 'Fatal (Y/N)', 'Species '], sdf2)

#### Vamos a limpiar las col 'Injury' y 'Fatal'

#### Si bien ambas columnas poseen datos sobre la letalidad, la columna 'Sepecies' es la que mas datos nulos posee. Comenzaremos limpiando esa columna

In [213]:
sdf2['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [214]:
sdf2['Injury'].isna().value_counts()

True     19449
False     6274
Name: Injury, dtype: int64

In [216]:
sdf2.dropna(inplace=True)

In [167]:
sdf2['Identified Specie'] = sdf2['Species '].apply(cf.filtracion_especies)

#### Vamos a armar dos data frames, uno con las muertes y otro que considere todos los ataques

#### ATAQUES POR ESPECIE

In [168]:
ataques = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'N'])

In [169]:
ataques = pd.DataFrame(sdf2['Identified Specie'].value_counts()).reset_index()

In [170]:
d = {'Identified Specie': 'Ataques', 'index':'Especie'}

In [171]:
ataques = ataques.rename(columns=d)

#### Como hay muchas categorias de tiburones con tan solo un registro, vamos a considerar solo las que tengan al menos 15 registros de ataque.

In [172]:
ataques = ataques[ataques['Ataques'] > 15]

In [173]:
ataques.shape

(26, 2)

In [174]:
display(ataques)

,Especie,Ataques
0,white shark,625
1,tiger shark,256
2,bull shark,175
3,nurse shark,97
4,whaler shark,65
5,reef shark,65
6,5' shark,65
7,4' shark,63
8,blacktip shark,62
9,small shark,54


In [175]:
ataques.to_csv('Ataques-especies',index=False)

#### LETALIDAD POR ESPECIE

#### Como la columna Fatal posee mas valores no nulos que Injury, potencialmente podemos obtener mas info sobre letalidad, vamos a prescindir de Injury. Vamos a quedarnos con los valores en donde hay un 'Y' en Fatal, en la col letalidad

In [176]:
sdf2.drop(columns=['Injury'], inplace=True)

In [177]:
letal = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'Y'])

In [178]:
letal2 = pd.DataFrame(letal['Identified Specie'].value_counts()).reset_index()

In [179]:
d = {'Identified Specie': 'Cantidad de Muertes', 'index':'Especie'}
letal2 = letal2.rename(columns=d)

#### Como podemos ver, a partir de la septima fila, los registros son muy, bajos, vamos a recortar el data frame para quedarnos con las especies que mas registros tienen

In [180]:
letal2.drop(range(8, 41), axis=0, inplace=True)

In [181]:
letal2.to_csv('especies-letalidad',index=False)

## HIPOTESIS III  Actividades/Ataques/Muertes

In [182]:
sdf3 = pd.DataFrame(sharks)

#### Conservamos las columnas que nos interesan

In [183]:
cf.keep_columns(['Activity', 'Fatal (Y/N)'], sdf3)

In [184]:
sdf3['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [185]:
sdf3['Activity'].isna().value_counts()

True     19965
False     5758
Name: Activity, dtype: int64

#### Se puede observar que la cantidad de nulos es similar para ambas variables.

In [186]:
sdf3.dropna(inplace=True)

#### Vamos a homogeneizar las columnas de actividades y fatalidades y eliminaremos los valores nulos

In [187]:
sdf3['Actividades'] = sdf3['Activity'].apply(cf.filtracion_actividades)

In [188]:
cf.keep_columns(['Actividades', 'Fatal (Y/N)'], sdf3)

In [189]:
sdf3.dropna(inplace=True)

#### Definimos un df para registrar la cantidad de ataques no letales,  por actividad

In [190]:
act = pd.DataFrame(sdf3[sdf3['Fatal (Y/N)'] == 'N'])

In [191]:
act = pd.DataFrame(sdf3['Actividades'].value_counts()).reset_index()

In [192]:
f = {'Actividades': 'Cantidad de Ataques', 'index':'Actividad'}

In [193]:
act = act.rename(columns=f)

In [194]:
act.head()

,Actividad,Cantidad de Ataques
0,surfing,1043
1,swimming,996
2,fishing,674
3,diving,471
4,spearfishing,379


#### Vamos a quedarnos con las actividades que registren mas de 20 ataques

In [195]:
act = act[act['Cantidad de Ataques'] > 20]

#### Definimos un df para registrar la cantidad de muertes por actividad

In [196]:
act_let = pd.DataFrame(sdf3[sdf3['Fatal (Y/N)'] == 'Y'])

In [197]:
act_let = pd.DataFrame(act_let['Actividades'].value_counts()).reset_index()

In [198]:
e = {'Actividades': 'Cantidad de Muertes', 'index':'Actividad'}

In [199]:
act_let = act_let.rename(columns=e)

#### Vamos a quedarnos con las actividades que registren mas de 20 muertes

In [200]:
act_let = act_let[act_let['Cantidad de Muertes'] > 20]

### vamos a unir ambos df

In [201]:
j = act.set_index('Actividad').join(act_let.set_index('Actividad'))

In [202]:
j.dropna(inplace=True)

In [203]:
j.to_csv('actividades-ataques.csv')